# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [32]:
from azureml.core import Workspace, Experiment, Dataset, Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
import logging
from azureml.train.automl import AutoMLConfig
from azureml.automl.core.forecasting_parameters import ForecastingParameters
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split

## Dataset

### Overview

This dataset contains information about sales of stores from a Retail Company, like Walmart.The dataset contains historical weekly sales values(target column) and other
supporting variables for that period like:
1. Store identifier
2. average temperature in the week 
3. whether or not there was a holiday during the week
4. fuel price
5. Consumer Price Index(CPI)
6. Unemployment rate 

The goal of this task is to use historical data to forecast sales numbers for the next four weeks(month). These predictions are going to support
finance and business people in the company to manage the store's inventory. <br>
This dataset comes from Kaggle and further details about it can be found [here.](https://www.kaggle.com/datasets/asahu40/walmart-data-analysis-and-forcasting) 

### Get data.
I have downloaded the dataset from Kaggle and uploaded it to this notebook's working directory. <br>Now I am going to import it with the help of the
Dataset class.


In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [5]:
datastore = Datastore.get(ws, datastore_name='workspaceworkingdirectory')

In [6]:
data = Dataset.Tabular.from_delimited_files(path=(datastore, "Users/hualcosa/nd00333-capstone/data/Walmart Data Analysis and Forcasting.csv"))
type(data)

azureml.data.tabular_dataset.TabularDataset

In [7]:
# getting data as pandas dataframe for local experiment
df = data.to_pandas_dataframe()
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,2010-02-12,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,2010-02-19,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,2010-02-26,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,2010-03-05,1554806.68,0,46.50,2.625,211.350143,8.106


In [ ]:
# TODO - reformat target column and windows (MAYBE)

In [18]:
# registering dataset so It can be used for automl experiment
data.register(ws, name="sales_forecasting", description="capstone project dataset")


{
  "source": [
    "('workspaceworkingdirectory', 'Users/hualcosa/nd00333-capstone/data/Walmart Data Analysis and Forcasting.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "a8489742-86d6-4cec-b52b-6098c3e55e4a",
    "name": "sales_forecasting",
    "version": 1,
    "description": "capstone project dataset",
    "workspace": "Workspace.create(name='capstone-project', subscription_id='d2706c67-acfc-4bd3-9067-3ff6ac190bc9', resource_group='capstone-project')"
  }
}

### Now that we have registered the dataset, it appears as an data asset, and can be used as an input source to the automl experiment.


In [8]:
# Choose a name for your CPU cluster
cpu_cluster_name = "capstone-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                            min_nodes=1,
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned



## AutoML Configuration
In order to run the the AutoML experiment, we set the following parameters:
- compute_target where the experiment is going to run. In our case, the compute cluster, we just created
- primary metric: The metric we want to optimize. Since we're dealing with a time series forecasting problem, Normalized root mean squared error is a great pick
- experiment_timeout_minutes: Maximum time that the experiment can run. I want the experiment to run for 60 minutes maximum.
- enable_early_stopping: Set it to True to allow the training iteration to prematurely end if the model scores are not improving
- n_cross_validations and cv_step_size: cross validation parameters. Set it to "auto" so automl job can investigate how to best split the data to perform cross validation
- enable_voting_ensenble and enable_dnn: Set it to true to consider voting ensemble models as well as Dense Neural Networks when searching for the best model.
- Verbosity: set logging verbosity to INFO
- Forecasting parameters: Object containing info about the forecasting job that needs to be performed. In our specifc case, it specifies what is the name of the time column, <br>
what is the forecast horizon(4 weeks) and what column(s) identify the time series

In [18]:
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.9,0,42.31,2.572,211.096358,8.106


In [55]:
forecasting_parameters = ForecastingParameters(time_column_name='Date', 
                                               forecast_horizon=4,
                                               time_series_id_column_names='Store')
                                               
automl_settings = {'compute_target': cpu_cluster,
                    'primary_metric':'normalized_root_mean_squared_error',
                    'experiment_timeout_minutes': 60,
                    'enable_early_stopping': True,
                    'n_cross_validations': "auto", # Could be customized as an integer
                    'cv_step_size' : "auto", # Could be customized as an integer
                    'enable_voting_ensemble': True,
                    'enable_dnn': True,
                    'verbosity': logging.INFO,
                    'forecasting_parameters': forecasting_parameters}

automl_config = AutoMLConfig(
                             task='forecasting',
                             training_data=data,
                             label_column_name='Weekly_Sales',
                             **automl_settings
                             )

In [56]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on capstone-cluster with default configuration
Running on remote compute: capstone-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project,AutoML_5e8945c3-9c9b-4df1-8b84-ecd7ff691c6b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Time Series ID detection
STATUS:       PASSED
DESCRIPTION:  The data set was analyzed, and no duplicate time index were detected.
              Learn more about time-series forecasting configurations: https://aka.ms/AutomatedMLForecastingConfiguration

********************************************************************************************

TYPE:         Frequency detection
STATUS:       PASSED
DESCRIPTION:  The time series was analyzed, all data points are aligned with detected frequency.
              Learn more about data preparation for time-series forecasting: https://aka.ms/AutomatedMLDataPreparation

********************************************************************************************

TYPE:         Data size v

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [58]:
RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
